# Multi-instrument Data Analysis Workflows with the SunPy Ecosystem

In this notebook, we'll look at how to use the SunPy Ecosystem to analyze data from multiple instruments, including SDO/AIA, EUI/HRI, EUI/FSI, SPICE, and the VISP instrument on DKIST.
The goal is to show how the SunPy Ecosystem allows you to easily work with these disparate datasources using the same set of tools.
The data we'll be looking is from 24 October 2022, close to the *Solar Orbiter* perihelion.
During this period, DKIST was pointed to coincide with this perihelion.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.visualization import ImageNormalize, AsinhStretch

from sunpy.net import Fido, attrs
from sunpy.coordinates import get_earth, get_horizons_coord

import sunraster.instr.spice

## Searching for and Downloading Data with `Fido`

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.subplot(projection='polar')

# Plot the Sun
ax.plot(0, 0, marker='o', markersize=20, label='Sun', color='yellow')

# Plot the satellite locations
obstime = "2022-10-24 19:00"
for body_name in ['Earth', 'SDO', 'Solar Orbiter']:
    if body_name == 'Earth':
        body = get_earth(obstime)
    else:
        body = get_horizons_coord(body_name, time=obstime)
    p, = ax.plot(body.lon.to('rad'), body.radius.to(u.AU), 'o', label=body_name)
    ax.plot([body.lon.to_value('rad'), 0], [body.radius.to_value(u.AU), 0], ls='--', color=p.get_color())

ax.set_theta_zero_location("S")
ax.set_rlabel_position(90)
ax.set_rlim(0, 1.3)
ax.legend()

### SDO/AIA

In [ ]:
time_range = attrs.Time("2022-10-24T18:55", "2022-10-24T19:35")

In [ ]:
Fido.search(
    time_range,
    attrs.Instrument.aia,
    attrs.Wavelength(171*u.Angstrom),
    attrs.Sample(10*u.minute)
)

### Solar Orbiter--EUI and SPICE

In [ ]:
import sunpy_soar

In [ ]:
Fido.search(
    time_range,
    attrs.soar.Product('EUI-HRIEUV174-IMAGE') | attrs.soar.Product('EUI-FSI174-IMAGE') | attrs.soar.Product('SPICE-N-RAS'),
    attrs.Level(2)
)

### DKIST

In [ ]:
import dkist.net

In [ ]:
Fido.search(
    time_range,
    attrs.Instrument('VISP')
)

### Combining them all...

In [ ]:
aia_query = attrs.Instrument.aia & attrs.Wavelength(171*u.Angstrom) & attrs.Sample(10*u.minute)

In [ ]:
solo_query = (attrs.soar.Product('EUI-HRIEUV174-IMAGE') | 
              attrs.soar.Product('EUI-FSI174-IMAGE') | 
              attrs.soar.Product('SPICE-N-RAS')) & attrs.Level(2)

In [ ]:
dkist_query = attrs.Instrument('VISP')

In [ ]:
q = Fido.search(time_range, aia_query | solo_query | dkist_query)

In [ ]:
q

In [ ]:
files = Fido.fetch(q, path='data/{instrument}')

## Visualizing Fields of View

In [ ]:
import sunpy.map
import dkist

In [ ]:
m_aia = sunpy.map.Map('data/AIA/aia_lev1_171a_2022_10_24t19_15_09_35z_image_lev1.fits')

In [ ]:
m_hri = sunpy.map.Map('data/EUI/solo_L2_eui-hrieuv174-image_20221024T191510172_V01.fits')

In [ ]:
m_fsi = sunpy.map.Map('data/EUI/solo_L2_eui-fsi174-image_20221024T191050177_V01.fits')

In [ ]:
raster_spice = sunraster.instr.spice.read_spice_l2_fits('data/SPICE/solo_L2_spice-n-ras_20221024T191303_V04_150995395-059.fits')
spice_window = raster_spice['N IV 765 - SH - Comp 8 ... Ne VIII 770 - LH - Comp 8 (Merged)'].apply_exposure_time_correction()

In [ ]:
visp = dkist.load_dataset("./data/VISP/VISP_L1_20221024T185745_BKEWK_updated.asdf")

The FSI field of view is quite large so let's zoom in on it a bit.

In [ ]:
m_fsi.peek()

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
m_fsi_zoom = m_fsi.submap(SkyCoord(-3000,-3000, unit='arcsec', frame=m_fsi.coordinate_frame),
                          top_right=SkyCoord(3000, 3000, unit='arcsec', frame=m_fsi.coordinate_frame))

We can then overlay field of view of HRI to show what feature on the disk HRI is looking at.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=m_fsi_zoom)
m_fsi_zoom.plot(axes=ax)
m_fsi_zoom.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(m_hri.dimensions),
    label='HRI',
    edgecolor='C0',
    lw=2,
    transform=ax.get_transform(m_hri.wcs),
)

Now, let's overlay the fields of view of our other instruments on top of this FSI image.

In [ ]:
visp_frame = visp.wcs.output_frame.frames[1].reference_frame
visp_space = visp[0, :, 500, :]
visp_corners = visp_space.wcs.pixel_to_world([0, visp_space.data.shape[1]-1],[0, visp_space.data.shape[0]-1])[0]

In [ ]:
m_spice = sunpy.map.Map(spice_window[0,51,:,:].data, spice_window[0,51,:,:].meta)

In [ ]:
import sunpy.visualization.drawing

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=m_fsi_zoom)
m_fsi_zoom.plot(axes=ax)
m_fsi_zoom.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(m_hri.dimensions),
    label='HRI',
    edgecolor='C0',
    lw=2,
    transform=ax.get_transform(m_hri.wcs),
)
m_fsi_zoom.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(m_spice.dimensions),
    label='SPICE',
    edgecolor='C1',
    lw=2,
    transform=ax.get_transform(m_spice.wcs),
)
m_fsi_zoom.draw_quadrangle(
    visp_corners,
    label="VISP",
    edgecolor='C2',
    lw=1,
    transform=ax.get_transform(visp_frame)
)
sunpy.visualization.drawing.limb(ax, m_aia.observer_coordinate, rsun=m_aia.rsun_meters, color='C3', lw=2, label='AIA limb')
ax.legend()

Similarly, we can overlay these same fields of view on our full-disk AIA image.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=m_aia)
m_aia.plot(axes=ax)
m_aia.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(m_hri.dimensions),
    label='HRI',
    edgecolor='C0',
    lw=2,
    transform=ax.get_transform(m_hri.wcs),
)
m_aia.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(m_spice.dimensions),
    label='SPICE',
    edgecolor='C1',
    lw=2,
    transform=ax.get_transform(m_spice.wcs),
)
m_aia.draw_quadrangle(
    visp_corners,
    label="VISP",
    edgecolor='C2',
    lw=1,
    transform=ax.get_transform(visp_frame)
)
m_aia.draw_grid(axes=ax)
sunpy.visualization.drawing.limb(ax, m_fsi.observer_coordinate, rsun=m_fsi.rsun_meters, color='C4', label='FSI limb', lw=2)
ax.legend()

## Reading in Cutouts

In [ ]:
center = SkyCoord(Tx=930*u.arcsec, Ty=630*u.arcsec, frame=m_hri.coordinate_frame)
width = 350*u.arcsec
height = 250*u.arcsec
loop_fov = center.spherical_offsets_by(width/[-2, 2], height/[-2, 2])

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=m_hri)
m_hri.plot(axes=ax)
x_lim = ax.get_xlim()
y_lim = ax.get_ylim()
m_hri.draw_quadrangle(
    loop_fov[0],
    top_right=loop_fov[1],
    label="HRI cutout",
)
m_hri.draw_quadrangle(
    [0,0]*u.pix,
    top_right=u.Quantity(m_spice.dimensions),
    label='SPICE FOV',
    edgecolor='C1',
    transform=ax.get_transform(m_spice.wcs),
)
m_hri.draw_quadrangle(
    visp_corners,
    label="VISP",
    edgecolor='C2',
    transform=ax.get_transform(visp_frame)
)
ax.plot_coord(center, marker='x', color='k', ls=' ')
ax.set_xlim(x_lim)
ax.set_ylim(y_lim)
plt.legend()

In [ ]:
import pathlib
import warnings

import astropy.io.fits
import astropy.wcs
from astropy.nddata import Cutout2D

from sunpy.coordinates import propagate_with_solar_surface

We only want to visualize the region around the loop so we don't want to wast time/memory reading in the parts of the image we won't use.

In [ ]:
hri_maps = []
for filename in sorted(pathlib.Path('./data/EUI/').glob('solo_L2_eui-hrieuv174-image_*.fits')):
    with astropy.io.fits.open(filename) as hdul:
        with warnings.catch_warnings():  # silence some astropy FITS warnings
            warnings.simplefilter('ignore', astropy.wcs.FITSFixedWarning)
            wcs = astropy.wcs.WCS(hdul[1].header)
        with propagate_with_solar_surface():  # transform with solar rotation
            cutout = Cutout2D(hdul[1].section,  # cutout from full-image
                              position=center,
                              size=(height, width),
                              wcs=wcs)
    hri_maps.append(sunpy.map.Map(cutout.data, cutout.wcs))  # create sunpy map

In [ ]:
hri_maps = sunpy.map.Map(hri_maps, sequence=True)

In [ ]:
ani = hri_maps.plot(cmap=m_hri.plot_settings['cmap'],
                    norm=ImageNormalize(vmin=5e2, vmax=1.75e4,
                                        stretch=m_hri.plot_settings['norm'].stretch))
ani.save('eui-hri-loops.mp4', fps=15, dpi=300)

In [ ]:
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="60%" controls>
      <source src="eui-hri-loops.mp4" type="video/mp4">
</video>
</div>""")

We can also examine how this loop structure varies in the AIA data.

In [ ]:
aia_maps = []
for filename in sorted(pathlib.Path('data/AIA').glob('*.fits')):
    m_aia_full = sunpy.map.Map(filename)
    with propagate_with_solar_surface():
        m_aia_cutout = m_aia_full.submap(hri_maps[0].bottom_left_coord, top_right=hri_maps[0].top_right_coord)
    aia_maps.append(m_aia_cutout)
aia_maps = sunpy.map.Map(aia_maps, sequence=True)

In [ ]:
ani = aia_maps.plot(cmap=m_aia.plot_settings['cmap'],
                    norm=ImageNormalize(vmin=150, vmax=2e3,
                                        stretch=m_aia.plot_settings['norm'].stretch))
ani.save('sdo-aia-loops.mp4', fps=2, dpi=300)

In [ ]:
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="60%" controls>
      <source src="sdo-aia-loops.mp4" type="video/mp4">
</video>
</div>""")

## Feature Identification

There is a clear loop structure present in the above HRI image.
We can construct a coordinate that traces this structure.
Here, we'll do this manually, but we could imagine doing this with a point-and-click tool or a loop tracing algorithm.

In [ ]:
traced_loop =  SkyCoord(
    Tx=[786, 809, 853, 895, 955, 998, 1025, 1035, 1040, 1038, 1029]*u.arcsec,
    Ty=[623, 662, 703, 716, 715, 697, 664, 628, 594, 567, 541]*u.arcsec,
    frame=hri_maps[0].coordinate_frame,
)

In [ ]:
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(projection=hri_maps[0].wcs)
hri_maps[0].plot(axes=ax, **m_hri.plot_settings)
ax.plot_coord(traced_loop)

### EUI HRI: Time-distance Analysis

Next, we can trace out this structure in each frame of our HRI image to create a stack plot or a "time-distance" plot.

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
traced_loop_pixelated = sunpy.map.pixelate_coord_path(hri_maps[0], traced_loop)
traced_loop_distance = traced_loop_pixelated.separation(traced_loop_pixelated[0])

In [ ]:
intensity_stack = []
for m in hri_maps:
    _traced_loop_pixelated = sunpy.map.pixelate_coord_path(m, traced_loop)
    _traced_loop_distance = _traced_loop_pixelated.separation(_traced_loop_pixelated[0])
    intensity = sunpy.map.sample_at_coords(m, _traced_loop_pixelated)
    f_interp = interp1d(_traced_loop_distance.to_value('arcsec'), intensity.value, fill_value='extrapolate', kind='linear')
    intensity_interp = f_interp(traced_loop_distance.to_value('arcsec'))
    intensity_stack.append(u.Quantity(intensity_interp, intensity.unit))
intensity_stack = u.Quantity(intensity_stack)

We'll then build an `NDCube` data structure with an axis for time and distance along the loop

In [ ]:
import astropy.time
from ndcube import NDCube
from ndcube.extra_coords import SkyCoordTableCoordinate, TimeTableCoordinate

In [ ]:
wcs = (SkyCoordTableCoordinate(traced_loop_pixelated,
                               physical_types=("custom:pos.helioprojective.lon", "custom:pos.helioprojective.lat")) & 
       TimeTableCoordinate(astropy.time.Time([m.date for m in hri_maps]), physical_types="time", names="time")
      ).wcs

In [ ]:
hri_time_distance = NDCube(intensity_stack, wcs)

In [ ]:
plt.figure(figsize=(20,6))
hri_time_distance.plot(plot_axes=('x','y'), cmap='sdoaia171', norm=ImageNormalize(stretch=AsinhStretch()), aspect=.2)
plt.colorbar()

### SPICE: Spectra along loop

What does this look like in the SPICE data?
We can overlay this same loop structure on top of the SPICE raster.

In [ ]:
fig = plt.figure(figsize=(8,10))
ax = fig.add_subplot(projection=spice_window[0,51,:,:].wcs)
spice_window[0,51,:,:].plot(axes=ax,
                            aspect='auto',
                            cmap='irissjiFUV',
                            norm=ImageNormalize(stretch=AsinhStretch()))
with propagate_with_solar_surface():
    pix = spice_window[0,51,:,:].wcs.world_to_pixel(traced_loop, traced_loop.obstime)
ax.plot(*pix)

In [ ]:
m_spice_adjust = m_spice.shift_reference_coord(-38*u.arcsec, 20*u.arcsec)

In [ ]:
fig = plt.figure(figsize=(8,10))
ax = fig.add_subplot(projection=m_spice_adjust)
m_spice_adjust.plot(axes=ax,
                    aspect='auto',
                    cmap='irissjiFUV',
                    norm=ImageNormalize(stretch=AsinhStretch()))
with propagate_with_solar_surface():
    pix = m_spice_adjust.wcs.world_to_pixel(traced_loop)
ax.plot(*pix)

Then, we can extract the spectra along each point of the loop we designated above.

In [ ]:
from ndcube.extra_coords import QuantityTableCoordinate

In [ ]:
# Use the adjusted map to extract the intensity
traced_loop_pixelated = sunpy.map.pixelate_coord_path(m_spice_adjust, traced_loop)
array_indices = m_spice_adjust.wcs.world_to_array_index(sunpy.map.pixelate_coord_path(m_spice_adjust, traced_loop_pixelated))
intensity = spice_window[0].data[:, array_indices[0], array_indices[1]]
# Construct the world coordinate system
wcs = (SkyCoordTableCoordinate(traced_loop_pixelated,
                               physical_types=("custom:pos.helioprojective.lon", "custom:pos.helioprojective.lat")) & 
       QuantityTableCoordinate(spice_window[0].axis_world_coords_values("em.wl").em_wl,
                               physical_types="em.wl")).wcs
# Build an NDCube
spice_intensity =  NDCube(intensity, wcs)

In [ ]:
fig = plt.figure(figsize=(6,8))
ax = spice_intensity.plot(aspect="auto", cmap='irissjiFUV', plot_axes=('x','y'))
wave = ax.coords[2]
wave.set_format_unit(u.AA)
wave.set_major_formatter("x.x")

### DKIST/VISP: Extracting a spectra

Finally, let's extract a VISP spectra for one of the points along our loop.

In [ ]:
fig = plt.figure(figsize=(10, 11))
visp.plot(fig=fig)

In [ ]:
from astropy.time import Time
from astropy.visualization import quantity_support
quantity_support()

In [ ]:
center_hgs = center.transform_to("heliographic_stonyhurst")

In [ ]:
visp_array_coords = visp[0,:,0,:].wcs.world_to_array_index(center, Time("2022-10-24T18:57:45.634"))

In [ ]:
fig = plt.figure(figsize=(11, 6))
ax = visp[0, visp_array_coords[0], :, visp_array_coords[1]].plot()
_ = ax.set_title(f"VISP Spectra at HGS lon={center_hgs.lon:2.2f} lat={center_hgs.lat:2.2f}")

In [ ]:
spice_array_coords = m_spice.wcs.world_to_array_index(center)

In [ ]:
visp_wave = visp.axis_world_coords("em.wl")[0]
spice_wave = spice_window.axis_world_coords("em.wl")[0].to(u.nm)

In [ ]:
fig, (ax, ax2) = plt.subplots(1, 2, facecolor='w', figsize=(11, 7))

ax.plot(spice_wave, spice_window[0].data[:, spice_array_coords[0], spice_array_coords[1]], color="C0")
ax2.plot(visp_wave, visp[0, visp_array_coords[0], :, visp_array_coords[1]].data, color="C1")

ax.set_xlim(np.min(spice_wave), np.max(spice_wave)-0.01*u.nm)
ax2.set_xlim(np.min(visp_wave), np.max(visp_wave))

ax.set_ylim(0.11, 0.25)

ax.set_ylabel(f"SPICE Intensity [{spice_window.unit}]")
ax2.set_ylabel(f"VISP Intensity [{visp.unit}]")

# hide the spines between ax and ax2
ax.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax.yaxis.tick_left()
ax.tick_params(labelright=False)
ax2.yaxis.tick_right()
ax2.tick_params(labelright=True)
ax2.yaxis.set_label_position("right")

d = .01  # how big to make the diagonal lines in axes coordinates
# arguments to pass plot, just so we don't keep repeating them
kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
ax.plot((1-d, 1+d), (-d, +d), **kwargs)
ax.plot((1-d, 1+d), (1-d, 1+d), **kwargs)

kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
ax2.plot((-d, +d), (1-d, 1+d), **kwargs)
ax2.plot((-d, +d), (-d, +d), **kwargs)

_ = fig.suptitle(f"SPICE and VISP spectra at HGS lon={center_hgs.lon:2.2f} lat={center_hgs.lat:2.2f}")
fig.tight_layout()